In [1]:
import pandas as pd
import random
import sys
sys.path.append('/home/climaco/Aptamers/src')
import time
from APTamers import  Aptamer_Fold
from utils import create_batches
from tqdm import tqdm

In [11]:
# Specify the path to your CSV file
csv_file_path = '/home/climaco/Aptamers/SRTN.csv'
pcsv_file_path = '/home/climaco/Aptamers/pSRTN.csv'
times  = 10
df =  pd.concat([create_batches(csv_file_path, times), create_batches(pcsv_file_path, times)], ignore_index=False)
df = df.drop_duplicates(subset=['sequence'])
df = df.reset_index(drop=True)

In [10]:

def fold_structure( strn):
    APT = Aptamer_Fold(l_fix = 4)
    APT.fit_fold( sequence=strn,  n_tmpl=4, l_fix=8 , filters = False)
    return APT.structures_DB, APT.structures

structures_DB = []
structures = []
start = time.time()
for i in tqdm(range(len(df['sequence']))):
    stra  = fold_structure( df['sequence'][i])
    structures_DB.append(stra[0])
    structures.append(stra[1])
print(time.time()- start) 
    


  3%|▎         | 7/273 [00:00<00:07, 34.05it/s]

100%|██████████| 273/273 [00:05<00:00, 46.36it/s]

5.8917555809021


In [ ]:
def base_pair_distance(structure1, structure2):
    bp_set1 = set(structure1.pairs())
    bp_set2 = set(structure2.pairs())
    common_pairs = bp_set1.intersection(bp_set2)
    distance = len(bp_set1.union(bp_set2)) - len(common_pairs)
    return distance

In [5]:
    structures_DB[2]

'((((...((((.......)).......................))...))))'

In [2]:


def remove_spaces_and_convert_to_upper(input_string, remove_first = 5):
    #input_string = input_string.upper()
    input_string = input_string.replace(" ", "")
    return input_string[remove_first:]


def substitute_random_chars(input_string, d = 4 , l= 8):
    
    '''
    d is the number of nucleotides we want to swap in  the string
    
    l i s the nuber of intial/final nucleatides that are fixed
    
    '''
    
    if len(input_string) <= 2:  # If string length is less than or equal to 2, cannot perform substitution
        return input_string

    # Number of elements to exclude from the start and end
    length = len(input_string)
    if length <= 2 * l:  # If string length is too small to perform substitution
        return input_string

    # Selecting characters between the first l and last l elements
    substr_start = l
    substr_end = length - l-1
    substr = input_string[substr_start:substr_end]

    # Randomly selecting d characters to substitute
    substitute_indices = random.sample(range(len(substr)), min(d, len(substr)))

    # Substituting selected characters with random characters from set (A, C, T, G)
    substitution_set = ['A', 'C', 'T', 'G']
    substr_list = list(substr)
    for index in substitute_indices:
        substr_list[index] = random.choice(substitution_set)

    # Reconstructing the string with substitutions
    output_string = input_string[:substr_start] + ''.join(substr_list) + input_string[substr_end:]
    return output_string


def change_name( input_sequence, seed ):
    
    return input_sequence + f'_seed_{seed}'

def create_batches(csv_file_path, times):
    
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file_path)
    df['sequence']= df['sequence'].apply(remove_spaces_and_convert_to_upper)
    sd = df

    for i in range( times ):
        random.seed(i)
        cd = df.copy()
        cd['sequence']= cd['sequence'].apply(substitute_random_chars)
        cd['name'] = cd['name'].apply(change_name, args=(i,))
        cd['times'] = 0
        sd = pd.concat([sd, cd], ignore_index=True)
    
    return sd.drop_duplicates(subset=['sequence'])